In [1]:
#!pip install ultralytics

In [1]:
import ultralytics
ultralytics.__version__

'8.3.48'

In [1]:
import cv2
import pandas as pd
from ultralytics import YOLO
from tracker import*
import os
import time

model=YOLO('yolov8s.pt')

In [2]:
class_list = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

In [4]:
count = 0
tracker = Tracker()
down = {}
up = {}
counter_down = []
counter_up = []

In [5]:
white_line_y = 128
red_line_y = 198
blue_line_y = 268
green_line_y = 368
offset = 6

In [6]:
cap=cv2.VideoCapture('highway.mp4')

In [7]:
# Create a folder to save frames
if not os.path.exists('detected_frames'):
    os.makedirs('detected_frames')

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 20.0, (1020, 500))

In [ ]:
while True:
    ret, frame = cap.read()
    if not ret:
        break
    count += 1
    # if count % 2 != 0:
    #     continue
    frame = cv2.resize(frame, (1020, 500))

    results = model.predict(frame)
    a = results[0].boxes.data
    a = a.detach().cpu().numpy()
    px = pd.DataFrame(a).astype("float")
    list = []

    for index, row in px.iterrows():
        x1 = int(row[0])
        y1 = int(row[1])
        x2 = int(row[2])
        y2 = int(row[3])
        d = int(row[5])
        c = class_list[d]
        if 'car' in c:
            list.append([x1, y1, x2, y2])
    bbox_id = tracker.update(list)

    for bbox in bbox_id:
        x3, y3, x4, y4, id = bbox
        cx = int(x3 + x4) // 2
        cy = int(y3 + y4) // 2

        if red_line_y<(cy+offset) and red_line_y > (cy-offset):
           down[id]=time.time()   # current time when vehichle touch the first line
        if id in down:
          
           if blue_line_y<(cy+offset) and blue_line_y > (cy-offset):
             elapsed_time=time.time() - down[id]  # current time when vehicle touch the second line. Also we a re minusing the previous time ( current time of line 1)
             if counter_down.count(id)==0:
                counter_down.append(id)
                distance = 10 # meters - distance between the 2 lines is 10 meters
                a_speed_ms = distance / elapsed_time
                a_speed_kh = a_speed_ms * 3.6  # this will give kilometers per hour for each vehicle. This is the condition for going downside
                cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
                cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255, 0), 2)  # Draw bounding box
                cv2.putText(frame,str(id),(x3,y3),cv2.FONT_HERSHEY_COMPLEX,0.6,(255,255,255),1)
                cv2.putText(frame,str(int(a_speed_kh))+'Km/h',(x4,y4 ),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)

                
        #####going UP#####     
        if blue_line_y<(cy+offset) and blue_line_y > (cy-offset):
           up[id]=time.time()
        if id in up:

           if red_line_y<(cy+offset) and red_line_y > (cy-offset):
             elapsed1_time=time.time() - up[id]
             # formula of speed= distance/time  (distance travelled and elapsed time) Elapsed time is It represents the duration between the starting point and the ending point of the movement.
             if counter_up.count(id)==0:
                counter_up.append(id)      
                distance1 = 10 # meters  (Distance between the 2 lines is 10 meters )
                a_speed_ms1 = distance1 / elapsed1_time
                a_speed_kh1 = a_speed_ms1 * 3.6
                cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
                cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255, 0), 2)  # Draw bounding box
                cv2.putText(frame,str(id),(x3,y3),cv2.FONT_HERSHEY_COMPLEX,0.6,(255,255,255),1)
                cv2.putText(frame,str(int(a_speed_kh1))+'Km/h',(x4,y4),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)



    
    text_color = (0, 0, 0)  # Black color for text
    yellow_color = (0, 255, 255)  # Yellow color for background
    red_color = (0, 0, 255)  # Red color for lines
    blue_color = (255, 0, 0)  # Blue color for lines

    cv2.rectangle(frame, (0, 0), (250, 90), yellow_color, -1)

    cv2.line(frame, (172, 198), (774, 198), red_color, 2)
    cv2.putText(frame, ('Red Line'), (172, 198), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

    cv2.line(frame, (172, 198), (774, 198), red_color, 2)
    cv2.putText(frame, ('Red Line'), (172, 198), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

    cv2.line(frame, (8, 268), (927, 268), blue_color, 2)
    cv2.putText(frame, ('Blue Line'), (8, 268), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

    cv2.putText(frame, ('Going Down - ' + str(len(counter_down))), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)
    cv2.putText(frame, ('Going Up - ' + str(len(counter_up))), (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

    # Save frame
    frame_filename = f'detected_frames/frame_{count}.jpg'
    cv2.imwrite(frame_filename, frame)

    out.write(frame)

    cv2.imshow("frames", frame)
    if cv2.waitKey(1) & 0xFF == 27:
    #if cv2.waitKey(0) & 0xFF == 27:
        break

cap.release()
out.release()
cv2.destroyAllWindows()

In [3]:
# import cv2
# import time
# import os
# import pandas as pd
# from tracker import Tracker

# count = 0
# tracker = Tracker()
# down = {}
# up = {}
# counter_down = []
# counter_up = []

# cap = cv2.VideoCapture('highway.mp4')

# # Set line positions
# red_line_y = 198
# blue_line_y = 268
# offset = 6

# # Create folder to save frames
# if not os.path.exists('detected_frames'):
#     os.makedirs('detected_frames')

# # Initialize video writer
# fourcc = cv2.VideoWriter_fourcc(*'XVID')
# out = cv2.VideoWriter('output.avi', fourcc, 20.0, (1020, 500))

# # Process video frames
# while True:
#     ret, frame = cap.read()
#     if not ret:
#         break
#     count += 1
#     frame = cv2.resize(frame, (1020, 500))  # Resize to reduce frame size
    
#     # Process every other frame
#     if count % 2 != 0:
#         continue

#     # Predict with model (adjust the model call if needed)
#     results = model.predict(frame)
#     a = results[0].boxes.data
#     a = a.detach().cpu().numpy()
#     px = pd.DataFrame(a).astype("float")
#     detected_objects = []

#     # Filter detected cars
#     for index, row in px.iterrows():
#         x1, y1, x2, y2, _, c = row[0], row[1], row[2], row[3], row[5], class_list[int(row[5])]
#         if 'car' in c:
#             detected_objects.append([x1, y1, x2, y2])
    
#     # Update tracker
#     bbox_id = tracker.update(detected_objects)

#     for bbox in bbox_id:
#         x3, y3, x4, y4, id = bbox
#         cx = int(x3 + x4) // 2
#         cy = int(y3 + y4) // 2

#         # Check crossing red line (going down)
#         if red_line_y < (cy + offset) and red_line_y > (cy - offset):
#             down[id] = time.time()
#         if id in down:
#             if blue_line_y < (cy + offset) and blue_line_y > (cy - offset):
#                 elapsed_time = time.time() - down[id]
#                 if id not in counter_down:
#                     counter_down.append(id)
#                     distance = 10  # meters
#                     a_speed_ms = distance / elapsed_time
#                     a_speed_kh = a_speed_ms * 3.6
#                     cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)
#                     cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255, 0), 2)
#                     cv2.putText(frame, str(id), (x3, y3), cv2.FONT_HERSHEY_COMPLEX, 0.6, (255, 255, 255), 1)
#                     cv2.putText(frame, str(int(a_speed_kh)) + 'Km/h', (x4, y4), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)

#         # Check crossing blue line (going up)
#         if blue_line_y < (cy + offset) and blue_line_y > (cy - offset):
#             up[id] = time.time()
#         if id in up:
#             if red_line_y < (cy + offset) and red_line_y > (cy - offset):
#                 elapsed1_time = time.time() - up[id]
#                 if id not in counter_up:
#                     counter_up.append(id)
#                     distance1 = 10  # meters
#                     a_speed_ms1 = distance1 / elapsed1_time
#                     a_speed_kh1 = a_speed_ms1 * 3.6
#                     cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)
#                     cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255, 0), 2)
#                     cv2.putText(frame, str(id), (x3, y3), cv2.FONT_HERSHEY_COMPLEX, 0.6, (255, 255, 255), 1)
#                     cv2.putText(frame, str(int(a_speed_kh1)) + 'Km/h', (x4, y4), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)

#     # Draw lines and info
#     cv2.line(frame, (172, red_line_y), (774, red_line_y), (0, 0, 255), 2)  # Red line
#     cv2.putText(frame, 'Red Line', (172, red_line_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
#     cv2.line(frame, (8, blue_line_y), (927, blue_line_y), (255, 0, 0), 2)  # Blue line
#     cv2.putText(frame, 'Blue Line', (8, blue_line_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)

#     cv2.putText(frame, f'Going Down - {len(counter_down)}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
#     cv2.putText(frame, f'Going Up - {len(counter_up)}', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)

#     # Save frame (optional)
#     # cv2.imwrite(f'detected_frames/frame_{count}.jpg', frame)
#     # out.write(frame)

#     cv2.imshow("Frames", frame)
#     if cv2.waitKey(1) & 0xFF == 27:
#         break

# cap.release()
# out.release()
# cv2.destroyAllWindows()



0: 320x640 8 cars, 2 trucks, 122.0ms
Speed: 2.1ms preprocess, 122.0ms inference, 1.3ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 6 cars, 2 trucks, 84.4ms
Speed: 3.0ms preprocess, 84.4ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 2 trucks, 92.4ms
Speed: 2.3ms preprocess, 92.4ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 2 trucks, 90.3ms
Speed: 0.5ms preprocess, 90.3ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 7 cars, 2 trucks, 89.3ms
Speed: 0.0ms preprocess, 89.3ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 10 cars, 1 truck, 89.9ms
Speed: 0.0ms preprocess, 89.9ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 9 cars, 1 truck, 85.1ms
Speed: 0.0ms preprocess, 85.1ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 9 cars, 2 trucks, 85.0ms
Speed: 4.9ms preprocess,

error: OpenCV(4.10.0) :-1: error: (-5:Bad argument) in function 'rectangle'
> Overload resolution failed:
>  - Can't parse 'pt1'. Sequence item with index 0 has a wrong type
>  - Can't parse 'pt1'. Sequence item with index 0 has a wrong type
>  - Can't parse 'rec'. Expected sequence length 4, got 2
>  - Can't parse 'rec'. Expected sequence length 4, got 2
